In [17]:
# Kütüphanelerin yüklenmesi
import pandas as pd
import numpy as np

# Veriyi yükleme ve hazırlama
file_path = 'Emre_YONTUCU_ticket.csv'
data = pd.read_csv(file_path)

# Gerekli sütunları uygun formatlara dönüştürme
data['IslemZamanı'] = pd.to_datetime(data['IslemZamanı'], errors='coerce')
data['KartBakiyesi'] = data['KartBakiyesi'].str.replace(',', '.').astype(float)
data['IslemTutarı'] = data['IslemTutarı'].str.replace(',', '.').astype(float)

# Günlük harcama sıklığı hesaplama
spending_frequency = data['IslemZamanı'].dt.date.value_counts().sort_index()

# Harcama sıklığını görüntüleme
print("Daily Spending Frequency:\n")
print(spending_frequency)

# Hafta içi ve hafta sonu sınıflandırması
data['day_of_week'] = data['IslemZamanı'].dt.day_name()
data['is_weekend'] = data['day_of_week'].isin(['Saturday', 'Sunday'])

# Hafta içi ve hafta sonu harcamalarını ayırma
weekday_spending = data[data['is_weekend'] == False]['IslemTutarı']
weekend_spending = data[data['is_weekend'] == True]['IslemTutarı']

# Hipotez testi (tamamen numpy ile)
def manual_ttest(data1, data2):
    """ İki bağımsız örnek için t-testi uygular """
    mean1, mean2 = np.mean(data1), np.mean(data2)  # Ortalama
    var1, var2 = np.var(data1, ddof=1), np.var(data2, ddof=1)  # Varyans
    n1, n2 = len(data1), len(data2)  # Veri boyutları

    # T-statistic hesaplama
    pooled_se = np.sqrt((var1 / n1) + (var2 / n2))
    t_stat = (mean1 - mean2) / pooled_se

    # P-value hesaplama (manuel olarak z-testi yaklaşımı ile)
    # Serbestlik derecesi hesaplanıyor
    df = ((var1 / n1 + var2 / n2) ** 2) / (((var1 / n1) ** 2) / (n1 - 1) + ((var2 / n2) ** 2) / (n2 - 1))
    # Z-testi yaklaşımı için normal dağılım kullanılır
    from math import erf, sqrt
    p_value = 2 * (1 - 0.5 * (1 + erf(abs(t_stat) / sqrt(2))))

    return t_stat, p_value

# Numpy ile T-testi
t_stat, p_value = manual_ttest(weekday_spending.dropna(), weekend_spending.dropna())

# Hipotez testi sonuçları
print("Hypothesis Testing Results:\n")
print(f"T-statistic: {t_stat}")
print(f"P-value: {p_value}")
if p_value < 0.05:
    print("Conclusion: There is a significant difference between weekday and weekend spending.")
else:
    print("Conclusion: There is no significant difference between weekday and weekend spending.")

# Rapor satırlarını oluşturma
report_lines = [
    "Spending Frequency and Hypothesis Testing Report\n",
    "Date\t\tNumber of Transactions\n"
]
report_lines.extend([f"{date}\t{count}\n" for date, count in spending_frequency.items()])

# Hipotez testi sonuçlarını ekleme
report_lines.append("\nHypothesis Testing:\n")
report_lines.append(f"T-statistic: {t_stat}\n")
report_lines.append(f"P-value: {p_value}\n")
if p_value < 0.05:
    report_lines.append("Conclusion: There is a significant difference between weekday and weekend spending.\n")
else:
    report_lines.append("Conclusion: There is no significant difference between weekday and weekend spending.\n")

# Raporu bir metin dosyasına kaydetme
output_path = 'Spending_Frequency_Hypothesis_Testing_Report_Numpy_Only.txt'
with open(output_path, 'w') as report_file:
    report_file.writelines(report_lines)

print(f"Report saved to {output_path}")


Daily Spending Frequency:

IslemZamanı
2023-10-31    1
2023-11-13    1
2023-11-14    1
2023-11-15    1
2023-11-20    1
             ..
2024-11-15    1
2024-11-18    1
2024-11-19    1
2024-11-25    1
2024-11-26    1
Name: count, Length: 89, dtype: int64
Hypothesis Testing Results:

T-statistic: -0.28838438381078685
P-value: 0.7730525201247296
Conclusion: There is no significant difference between weekday and weekend spending.
Report saved to Spending_Frequency_Hypothesis_Testing_Report_Numpy_Only.txt
